In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import pickle
import multiprocessing
from multiprocessing import Process, Value, Lock, Pool
import gc
PATH = '/home/anurag/practice/datasets/UrduWikipediaArticles/'
mapping_dict = {}

In [2]:
with open('all_urdu_wikipedia_links.pkl', 'rb') as f:
    all_urls = pickle.load(f)

In [3]:
len(all_urls)

345069

In [ ]:
# for counter, url in enumerate(all_urls):
#     if counter < 3691:
#         continue
def parallel_fetch(offset,urls):
    print(multiprocessing.current_process())
    for counter,url in enumerate(urls):
        html_doc = ''
        try:
            with urlopen(url) as response:
                for line in response:
                    line = line.decode('utf-8')
                    html_doc = html_doc + line.replace('\n','')
            soup = BeautifulSoup(html_doc, 'html.parser')
            print(soup.title.string)
            paras = soup.find_all('p')
            article = ''
            for para in paras:
                article = article + para.text + '\n'
            article = re.sub(r'\([^)]*\)', r'', article)
            article = re.sub(r'\[[^\]]*\]', r'', article)
            article = re.sub(r'<[^>]*>', r'', article)
            article = re.sub(r'^https?:\/\/.*[\r\n]*', '', article)
            article = article.replace(u'\ufeff','')
            article = article.replace(u'\xa0', u' ')
            article = article.replace('  ', ' ');
            article = article.replace(' , ', ', ');
    #         devanagari_nums = ('०','१','२','३','४','५','६','७','८','९')
    #         for c, n in enumerate(devanagari_nums):
    #             article = re.sub(n, str(c), article)
            filename = PATH + str(counter+offset) + '.pkl'
#             mapping_dict[filename] = soup.title.string
            with open(filename, 'wb') as f:
                pickle.dump(article,f)
            print("Saved " + filename)
            gc.collect()
        except:
            print('Failed')
            continue
    #     break

In [ ]:
done=0
total_news = len(all_urls)
cpus = multiprocessing.cpu_count()
chunk_size = total_news//cpus
procs = [Process(target=parallel_fetch, args=(cpu*chunk_size+done,all_urls[cpu*chunk_size+done:(cpu+1)*chunk_size]))for cpu in range(cpus)]

for p in procs: p.start()
for p in procs: p.join()